In [13]:
import sys

import pandas as pd
import numpy as np

sys.path.append('../src/')

from data.import_data_pks import hierarchize_data

In [14]:
data = pd.read_excel("../data/raw/Zeitreihen_Bund.xlsx", skiprows=14)
data = data.loc[:,[1,2,3,4,5,10]]
data.columns = [
    "key", "label", "year", "cases", "freq", "clearance"
]

In [16]:
hierarchize_data(data)

,key,label,year,cases,freq,clearance,level,parent
0,****00,Diebstahl insgesamt,1987,2790849,4564.7,27.800000,1,None
1,****00,Diebstahl insgesamt,1988,2655156,4335.8,28.700000,1,None
2,****00,Diebstahl insgesamt,1989,2571248,4166.3,28.900000,1,None
3,****00,Diebstahl insgesamt,1990,2692246,4295.3,30.200000,1,None
4,****00,Diebstahl insgesamt,1991,2874612,4422.4,28.700000,1,None
...,...,...,...,...,...,...,...,...
13695,899500,Sachbeschädigung durch Graffiti insgesamt,2018,97610,117.897365,19.000000,4,899000
13696,899500,Sachbeschädigung durch Graffiti insgesamt,2019,96244,115.929789,16.374008,4,899000
13697,899500,Sachbeschädigung durch Graffiti insgesamt,2020,109441,131.592315,15.938268,4,899000
13698,899500,Sachbeschädigung durch Graffiti insgesamt,2021,100558,120.928342,16.443247,4,899000
